In [ ]:
# !pip install openai

In [ ]:
import openai

openai.api_key = ''

In [ ]:
prompt = 'Qual é a capital do Brasil?'

In [ ]:
# # Deprecated
# openai.Completion.create(
#       model = 'text-davinci-003', # Modelo mais voltado ao complemento de texto
#       prompt=prompt, # prompt ==  informações passadas para a IA responder
#       temperature=0 # Nivel de aleatoriedade nas respostas
# )

### Modelo para enviar um request para a API do OpenAi
#### Versão API > 1.00

In [ ]:
#OK!
messages = [{
    "role":"user",
    "content": prompt

             }]

openai.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages= messages,
    temperature = 0
)

### encapsulando a rotina de mensagem

In [ ]:
def enviar_mensagem(message):
  messages = [{"role":"user", "content": message}]
  response = openai.chat.completions.create(model = 'gpt-3.5-turbo', messages = messages, temperature = 0)
  return response['choices'][0]['message']['content']

enviar_mensagem("Quem ganhou a copa do mundo de 2002?")

# Output: "O Brasil ganhou a Copa do Mundo de 2002."

### Usando format strings no prompt


In [ ]:
text = "O Brasil goleia Guiné por 4 a 1 no amistoso em Barcelona, na Espanha"

lang = 'Inglês'

In [ ]:
prompt = f''' Traduza para o {lang} o seguinte texto:

{text}
'''

In [ ]:
print(prompt)

#Output: 'Traduza para o Inglês o seguinte texto: \n \n O Brasil goleia Guiné por 4 a 1 no amistoso em Barcelona, na Espanha'

In [ ]:
enviar_mensagem(prompt)

#Output: 'Brazil beats Guinea 4-1 in friendly match in Barcelona, Spain.'

### Gerando tradução por palavra

In [ ]:
prompt_engineered = f''' Escreva o resultado e uma lista, com a palavra original e sua tradução:

{prompt}

'''

In [ ]:
resultado = enviar_mensagem(prompt_engineered)
print(resultado)

# Módulo 2

## Utilizando LangChain

### Models

In [ ]:
# !pip install langchain

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = 'sk-proj-mKUSgS3Mo7wxxfVA62mdT3BlbkFJiiAvNRmlSctfBA8ive9w'

In [ ]:
# Para o modelo do OpenAi é passado apenas o prompt

from langchain.llms import OpenAI

llm = OpenAI(temperature = 0.0)

In [ ]:
# Para o modelo do ChatOpenAi é passado uma lista de mensagens

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature= 0.0)


In [ ]:
prompt = ''' Traduza para o {lang} o seguinte texto:

{text}
'''

In [ ]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_template(prompt)

In [ ]:
chat_template

In [ ]:
chat_template.messages[0].prompt.template

#Output: ['lang', 'text']


In [ ]:
#modelo de ZERO SHOT TEMPLATE
prompt_engineered = chat_template.format_messages(
    lang='francês',
    text = '''Leclerc se chateia com Ferrari: "Dificultando nossa própria vida" \n
    Piloto de Mônaco sugeriu que equipe colocasse pneus secos para o Q2 na classificação do GP do Canadá, \n
    mas escuderia negou; do carro 16 acabou eliminado e obteve 11º tempo da sessão''')

In [ ]:
response = llm(prompt_engineered)
response.content

In [ ]:
# modelo de FEW SHOT TAMPLATE

exemplos = [
  {
    "question": "Quem viveu mais, Muhammad Ali ou Alan Turing?",
    "answer": """São necessárias perguntas de acompanhamento aqui? Sim.
    Acompanhamento: Qual era a idade de Muhammad Ali quando ele morreu?
    resposta: intermediária: Muhhamad Ali tinha 74 anos quando morreu.
    acompanhamento: Qual era a idade de Alan Turing quando ele morreu?
    Resposta intermediária: Alan Turing tinha 41 anos de idade quando morreu.
    Portanto, a resposta final é: Muhammad Ali"""
},
{
    "question": "Quando nasceu o fundador do craigslist?",
    "answer": """São necessárias perguntas de acompanhamento aqui? Sim.
    Acompanhamento: Quem foi o fundador da craigslist?
    resposta: intermediária: A craigslist foi fundada por Craig Newmark.
    acompanhamento: Quando Craig Newmark nasceu??
    Resposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952.
    Portanto, a resposta final é: 6 de dezembro de 1952"""

}]


In [ ]:
exemplo_prompt = PromptTemplate(input_variables= ["question", "answer"], template= "Pergunta: {question}\n{answer}")

In [ ]:
prompt = FewShotPromptTemplate(
    examples=exemplos,
    example_prompt = example_prompt,
    suffix = "Pergunta: {input}",
    input_variables = ["input"]
)

In [ ]:
print(prompt.format(input= "Quem ganhou mais prêmios Emmy, Mad Men ou Sopranos"))

In [ ]:
prompt = ChatPromptTemplate.from_template(prompt.format(input= "Quem ganhou mais prêmios Emmy, Mad Men ou Sopranos?"))

In [ ]:
resultado = llm(prompt.format_messages())

### Seletores

In [ ]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [ ]:
example_selector = LengthBasedExampleSelector(
    examples = exemplos,
    example_prompt = exemplo_prompt,
    max_length = 25,
)

In [ ]:
prompt = FewSHotPromptTemplate(
    example_selector = example_selector,
    example_prompt=exemplo_prompt,
    suffix="Pergunta: {input}",
    input_variables = ["input"]
)

In [ ]:
print(prompt.format(input="Quem ganhou mais prêmios Emmy, Mad Men ou Sopranos?"))

# Parsers

### Serve para formatar para algum formato, por exemplo JSON

In [ ]:
# modelo de output
desire_output = {
    "is_offensive": False,
    "main_point": "Harry is a great youtuber"
}

In [ ]:
# prompt (instruções de formatação)
template_with_output_format = """ Para o seguinte comentário de um vídeo do Youtube, extraia a seguintes informações
is_offensive: boolean - Se o comentário apresenta conteúdo ofensivo
main_point: String - O ponto principal do comentario, em cinco palavras ou menos.kwargs=

Escreva a resposta em formato JSON com as seguintes propriedades: is_offensive, main_point
Comentário: {comment}"""

In [ ]:
chat_template = ChatPromptTemplate.from_template(template_with_output_format)

prompt_engineered = chat_template.format_messages(comment = comment)

In [ ]:
print(request.content)


#Output:
# {
#     "is_offensive": false,
#     "main_point": "Harry cares for quality"
# }

# Output tipo String

### Transformando o tipo de saída para dict






In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
# Definindo o formato de resposta
offensive = ResponseSchema(name="offensive", description=" O comentário apresenta conteúdo ofensivo?.")
main_point = ResponseSchema(name="main_point", description=" Argumento central do comentário")

response_schemas = [offensive, main_point]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)


In [ ]:
format_instructions = output_parser.get_format_instructions()

In [ ]:
print(format_instructions)


#Output:
'''
json
{
  "offensive": string // o comentário apresenta conteúdo ofensivo?.,
  "main_point": string // Argumento central do comentário
}
'''

In [ ]:
template = """ Para o seguinte comentário de um vídeo do Youtube, extraia a seguintes informações
is_offensive: boolean - Se o comentário apresenta conteúdo ofensivo
main_point: String - O ponto principal do comentario, em cinco palavras ou menos.

{format_instructions}
Comentário: {comment}"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template = template)

messages = prompt.format_messages(comment = comment, format_instructions = format_instructions)


In [ ]:
response = llm(messages)

print(response.content)

#Output:
'''
json
{
  "offensive": string // o comentário apresenta conteúdo ofensivo?.,
  "main_point": string // Argumento central do comentário
}
'''


### Transformando em Dict

In [ ]:
output_dict = output_parser.parse(response.content)


#Output:

# {
#   "offensive": string // o comentário apresenta conteúdo ofensivo?.,
#   "main_point": string // Argumento central do comentário
# }


In [ ]:
output_dict.get('main_point')

#Output: 'Harry cares for quality content'

# Chains

### Chains (correntes) são encadeamentos de diversas chamadas

##### Ex: quando se pergunta para o chat-GPT onde se fica a estátua da liberdade, e em seguida pergunta-se quantos metros de altura ela tem.

##### O chat GPT vai reconhecer que você está perguntando a altura sobre o último assunto perguntado.

In [ ]:
from langchain.chains import LLMChain

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Crie uma lista com as atividades necessárias para produzir o seguinte produto: {product}"
)

In [ ]:
chain = LLMChain(llm = llm, prompt = prompt)
result = chain.run(product="Guitarra")

print(result)

#### Aplicando uma lista de produtos

In [ ]:
products = [{"product": "Frigideira"}, {"product": "Cafeteira"}]


result = chain.apply(products)


# Frigideira
print(result[0]['text'])

# Cafeteira
print(result[1]['text'])

#### usando SimpleSequencialChain

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
# Primeiro prompt
first_prompt = ChatPromptTemplate.from_template("Crie uma lista com as atividades necessárias para produzir o seguinte produto: {product}?")


# Chain 1

list_activities = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:
# segundo prompt

second_prompt = ChatPromptTemplate.from_template(
    "Dentre as seguintes atividades escreva qual delas é a mais importante e o motivo: {list}"
)

# Chain 2

most_important_activity = SimpleSequentialChain(chains=[list_activities, most_important_activity], verbose=True)

In [ ]:
# Combinando as Chains


combined_chain = SimpleSequentialChain(chains=[list_activities, most_important_activity], verbose=True)


result = combined_chain.run('Bootcamp de programação')

### Sequential Chain

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
# Determine o assunto da matéria
first_prompt = ChatPromptTemplate.from_template(
    "Escreva o nome da prncipal área do conhecimento que a seguinte matéria aborda: \n\n{news}"
)


# Chain 1 : Entrada= Matéria de jornal; Saída= Lista de tópicos

chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key='topic')

In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Cite o nome de um escritor famoso que escreve sobre a seguinte área do conhecimento:\n\n{topic}"
)

#chain 2 : Entrada = Lista de tópicos, Saída = Lista de pesquisadores
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key= "writer")

In [ ]:
# Escreva um resumo da matéria
third_prompt = ChatPromptTemplate.from_template(
    "Escreva um resumo da matéria a seguir: \n\n{news}"
)

#chain 3 : Entrada = Matéria de jornal, Saída = resumo
chain_three= LLMChain(llm=llm, prompt=third_prompt, output_key= "summary")

In [ ]:
# Escreva o resumo da matéria no estio do escritor escolhido
fourth_prompt = ChatPromptTemplate.from_template(
    "Reescreva o seguinte texto no estilo de {writer}: \n\n {summary}"
)

#chain 3 : Entrada = Escritor, Saída = resumo escrito por autor
chain_four= LLMChain(llm=llm, prompt=fourth_prompt, output_key= "writer_summary")

#### Sequenciando as chains

In [ ]:
writer_news_chain = SequentialChain(
    chains = [chain_one, chain_two, chain_three, chain_four],
    input_variables["news"],
    output_variables=["topic", "writer", "summary", "writer_summary"],
    verbose=True
)

In [ ]:
result = writer_news_chain('''
O resultado da Mega-Sena 2602 com prêmio de R$ 51.774.681,61 milhões foi divulgado neste sábado (17), em São Paulo, e uma aposta de Guarapari (ES) faturou sozinha a bolada.

O novo milionário(a) fez a aposta online pelo Internet Banking da Caixa e fez uma aposta simples, gastando R$ 5,00.

Os números sorteados hoje foram: 11 - 14 - 16 - 30 - 32 - 46

LEIA TAMBÉM:
- Quina de São João sorteia R$ 200 milhões: veja últimos resultados e mais curiosidades

111 apostas chegaram bem perto e acertaram cinco dezenas. Para cada uma delas a Caixa vai pagar R$ 44.054,71. Os 6.285 acertadores de quatro dezenas vão receber R$ 1.111,50 cada.
''')

In [ ]:
result["writer_summary"]

# Router Chain

### Funciona como uma espécie de if, else

#### Row Prompting
##### Se dá o nome Row Prompting quando se passa para a IA informações sobre quem ele é, funciona como fosse uma hipnose

In [ ]:
chef_template = '''Você é um chef de cozinha renomado e com muita experiência, além de ser extremamente didático.
Se você não sabe a resposta de algo, você responde que não sabe.
Responda a seguinte questão: {input}'''

mechanic_template = '''Você é um mecânico com experiência em todos os tipos de automóveis.
Se você não sabe a resposta de algo, você responde que não sabe.
Responda a seguinte questão: {input}'''

In [ ]:
prompt_infos = [
    {
        "name": "gastronomy",
        "description": "Recomendado para responder questões sobre gastronomia",
        "prompt_template": chef_template
    },
    {
        "name": "auto mechanic",
        "description": "Recomendado para responder questões sobre mecânica de automóveis",
        "prompt_template": mechanic_template
    }
]

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.prompts.router import MultiPromptChain
from langchain.prompts.router.llm_router import LLMRouterChain, RouterOutputParser

In [ ]:
destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p["name"]}: {p["description"]}" for p in prompt_infos]

destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
prompt_router_template = """Dado um texto selecione o prompt mais adequado. \
Você receberá os nomes dos prompts disponíveis e uma descrição do motivo pelo qual o prompt é mais adequado. \
Você também pode revisar o texto original se achar que a revisão levará a uma resposta melhor do modelo de linguagem.

<< FORMATAÇÃO >>
Retorne um trecho de código markdown com um objeto JSON formatado da seguinte maneira:
```json
{{{{
    "destination": string \ nome do prompt a ser usado ou "DEFAULT"
    "next_inputs": string \ uma versão modificada da entrada original
}}}}
```

LEMBRE-SE: "destination" DEVE ser um dos nomes de prompt candidatos  especificados abaixo OU pode ser "DEFAULT" se a entrada não for adequada a nenhum dos prompts candidatos.
LEMBRE-SE: "next_inputs" pode ser apenas a entrada original se você achar que nenhuma modificação é necessária.

<< PROMPTS CANDIDATOS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (lembre-se de incluir o ```json)>>"""

In [ ]:
router_template = prompt_router_template.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser= RouterOutputParser(),
)


router_chain = LLMRouterChain.from_llm(llm, router_prompt)


In [ ]:
chain = MultiPromptChain(router_chain = router_chain,
                         destination_chains = destination_chains,
                         default_chain = default_chain, verbose=True)

In [ ]:
#chef_template
result = chain.run("Qual é a receita de uma massa carbonara?")

print(result)

In [ ]:
# auto mechanic
result = chain.run("como posso trocar o pneu de um carro?")

print(result)

# Transformer Chain

### Transformer Chain é o nome que se dá quando existe um tratamento/processamento local

In [ ]:
from langchain.chains import TransformChain

In [ ]:
# Processamento local
def get_first_and_last_paragraph(inputs: dict) -> dict:
    text = inputs["text"]
    paragraphs = list(filter(None, text.split("\n\n")))
    shortened_text = paragraphs[0]
    shortened_text += paragraphs[-1]
    shortened_text = ''.join(shortened_text)
    return {"output_text": shortened_text}


transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=get_first_and_last_paragraph
)

In [ ]:
template = """Escreva um resumo do seguinte texto:

{output_text}

Resumo:"""

prompt = PromptTemplate(input_variables=["output_text"], template=template)

llm_chain = LLMChain(llm = llm, prompt=prompt)

In [ ]:
sequential_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain])

In [ ]:
sequential_chain.run('''
Jamiroquai é uma banda britânica de funk e acid jazz formada em 1992, liderada pelo cantor Jay Kay. A banda é popular no mundo todo e é um dos membros mais conhecidos do cenário acid jazz londrino do início dos anos 90, junto a outros grupos como Incognito, Galliano. Seu som é inspirado na música negra da década de 1970, e suas letras e conceitos visuais ocasionalmente lidam com o idealismo social e ambiental. Eles também se baseavam no rock, na música eletrônica e na música latina, e no palco se apresentavam com vários músicos da banda tocando ao vivo. Ao longo dos anos, Kay permaneceu consistentemente como líder em várias mudanças na formação.


O grupo vendeu mais de 35 milhões de álbuns no mundo inteiro e ganhou um Grammy em 1997.[1] Apesar de se ter estreado como uma banda de acid jazz, o grupo explorou outros géneros musicais ao longo dos anos.

O nome Jamiroquai deriva do nome da tribo nativa norte-americana Iroquoi, com a qual Jay Kay diz se identificar filosoficamente.[2]

Formação da banda
Jason "Jay" Kay começou a compor músicas para enviar a gravadoras. Entre elas, estava When You Gonna Learn?, escrita com 16 anos de idade.[3] De início, gravou-a em estúdio na Round House, em Camden.[4] Os produtores desta sessão alteraram-na e produziram-na com base nas tendências principais. Jay não gostou dos resultados e restaurou-os para a sua preferência, depois de uma disputa.[4] Em 1991, afiliou-se à Acid Jazz Records, depois de enviar uma gravação a cantar uma música dos Brand New Heavies.[5][6] De seguida, foi juntando membros para a banda, incluindo o seu amigo Wallis Buchanan, que tocava o didgeridoo.[4] Foi-lhe sugerido pelo seu agente que juntasse o teclista Toby Smith, mas não ficou convencido de início. Smith voltaria a encontrar a banda depois de um ato de suporte para os Brand New Heavies. Convenceu Kay a integrá-lo no grupo como teclista e co-compositor. A primeira música escrita pelos dois foi Too Young to Die.[4]

Sendo Jay Kay o líder e imagem da banda, muitos referem-se a ele como Jamiroquai. Além disso, é o único signatário do contrato com a discográfica sob o nome Jamiroquai.[7] Há boatos de que a banda é o resultado de uma audição falhada de Jay para vocalista dos Brand New Heavies. Contudo, estes rumores são desmentidos por eles.[8]


Primeiro single
O primeiro single da banda When You Gonna Learn? originou uma disputa entre gravadoras, sendo a Sony Soho Square vencedora, com quem assinaram um contrato de oito álbuns, uma proeza para um jovem que andava de skate, tinha uma paixão por roupas vintage e usava um chapéu esquisito.

Evolução da banda
O primeiro disco, Emergency On Planet Earth, de 1993, foi o álbum que mais cópias vendeu em menos tempo, desde os tempos de Faith, de George Michael.

The Return Of the Space Cowboy, de 1994, com os seus comentários satíricos e mordazes, é bastante agressivo, mas o sucesso da banda não ultrapassou as fronteiras britânicas. Entretanto, o álbum que deu a conhecer Jamiroquai a nível mundial, Travelling Without Moving, veio com um aviso preocupante, profético, sobre os perigos associados à engenharia biogenética. Tal álbum foi precedido pelo single Cosmic Girl, single de 4 faixas com remixes do êxito do grupo. O single vencedor de um Grammy e quatro prémios MTV Virtual Insanity, foi lançado no dia em que a ovelha Dolly nasceu. Este álbum conquistou a crítica e o público, tendo atingido o estatuto de platina em todo o mundo.

Logo após Virtual Insanity, surgiram desavenças durante a criação do álbum Synkronized. Com o álbum quase pronto, Stuart Zender deixou a banda, que, por motivos de direitos de autor, teve de refazer o álbum, de maneira a que Zender não constasse como co-autor. Boatos dizem que Stuart Zender se queixava de que Jay ganhava mais dinheiro do que os outros membros, e estava insatisfeito com isso. Há também uma especulação de que a última faixa do Synkronized, King for a Day, reflete a visão de Kay a respeito de Zender e sua ganância.

Depois de Synkronized, já sem a colaboração de Stuart Zender, o grupo colaborou em No Boundaries, um projeto que contou com vários artistas, como Pearl Jam e Alanis Morissette e cujas receitas reverteram para os refugiados do Kosovo.

Em 2001, lançam A Funk Odyssey, que trouxe o hit Love Foolosophy, seguido de quatro anos a preparar Dynamite.

Dynamite foi escrito e gravado em Espanha, Itália, Costa Rica, Escócia, Nova Iorque, Los Angeles e também no próprio estúdio de Jay, criado para o efeito na sua mansão em Buckinghamshire. Vale lembrar que apenas Jay Kay, Derrick McKenzie e Sola Akingbola constam da formação original do álbum.

No dia 16 de agosto de 2010, foi anunciado no site oficial da banda um novo álbum intitulado Rock Dust Light Star, previsto para ser lançado em novembro de 2010.

Em janeiro de 2017, a banda lança um vídeo promovendo o seu oitavo álbum, intitulado Automaton, que apresentou um primeiro single bem mais voltado para a música eletrónica; porém, outras canções lembraram os seus últimos trabalhos. Anunciaram também dez concertos na Ásia e Europa nesse ano. Começando por Tóquio, em maio, e terminando no Festival Sudoeste (Portugal), em agosto.

''')

# Chain APP - Chat Bot

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

##### A memória é run time, ou seja, se o servidor for desligado vai se perder o histórico

In [ ]:
memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [ ]:
conversation.predict(input="Olá! Como vai você?")

In [ ]:
# checar o que está na memória
memory.buffer

In [ ]:
# Checar o histórico
memory.load_memory_variables({})

## Tipos de memória

##### ConversationBufferMemory - Salva msgs
##### ConversationBufferWindowMemory - Salva últimas N msgs
##### ConversationTokenBufferMemory - Salva últimos N tokens
##### ConversationSummaryMemory - Cria um resumo da conversa


# Chain APP - Q&A (Question & Answer)

##### Incluir vários documentos pode ser um problema pela quantidade de tokens suportados
##### Uma forma de resolver isso é pela técnica de Vector Store
###### Vector Store é a técnica de divisão dos documentos em "blocos" representados de forma matemática

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
result = embeddings.embed_query('Olá! Tudo bem?')

result[:5]

In [ ]:
# !pip install wikipedia
# !pip install langchain[docarray]
# !pip install tiktoken

In [ ]:
from langchain.document_loaders import WikipediaLoader

In [ ]:
loader = WikipediaLoader(query="Seinfold", load_max_docs=10, lang='pt')

In [ ]:
docs = loader.load()

In [ ]:
list(map(lambda x : x.metadata['title'], docs))

In [ ]:
from langchain.vectorstores import DocArrayInMemorySearch

In [ ]:
db = DocArrayInMemorySearch.from_documents(
    docs,
    embeddings
)

In [ ]:
db.similarity_search_with_score('Quem é Larry David?')

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa_seinfeld = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={'k':1}),
    verbose=True
)

In [ ]:
result = qa_seinfeld.run('Quem é Larry David?')
result

In [ ]:
from langchain.evaluation.qa import QAGenerateChain

In [ ]:
generate_chain = QAGenerateChain.from_llm(llm)

In [ ]:
new_examples = generate_chain.apply_and_parse(
    [{"doc": t} for t in docs[:5]]
)

In [ ]:
new_examples[0]

In [ ]:
predictions = qa_seinfeld.apply(new_examples)

In [ ]:
from langchain.evaluation.qa import QAEvalchain

## Chain Types

#### Stuff - Agrega documentos relacionados no mesmo prompt
#### Map Reduce - Executa paralelamente uma chain para cada documento relacionado e usa os resultados como input para produzir a reposta final
#### Map Rerank - Executa paralelamente uma chain para cada documento relacionado e além de solicitar uma resposta, solicita um score e seleciona a reposta com score mais alto
#### Refine - Executa sequencialmente uma chain para cada documento relacionado e vai construindo a resposta final a cada iteração
---

# LangChain: Agents

In [ ]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [ ]:
tools = load_tools(["llm-math", "wikipedia"], llm=llm)

In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent= AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True
)

In [ ]:
agent("Quanto é 340 elevado na potência 0.23?")

#### Usando o interpretador Python

In [ ]:
agent = create_python_agent(
    llm,
    tool = PythonREPLTool(),
    verbose = True
)

In [ ]:
import numpy as np

random_array = np.random.randn(5)
print(random_array)

In [ ]:
result = agent(f"Ordene a seguinte lista de números: {random_array}")

### Criando um agente próprio

In [ ]:
from langchain.agents import tool
import requests

In [ ]:
# O langchain entende o que é pra fazer com a função através do comentário (documentação) da função

@tool # @tool para o langchain entender que é um agente
def cep(cep: str) -> str:
    """Retorna o endereço completo de um CEP em formato JSON, use-a para qualquer \
    pergunta relacionada à pesquisa de endereços. \
    A entrada deve ser sempre um código postal, \
    e essa função sempre retornará o endereço completo em formato JSON."""
    return requests.get(f'https://viacep.com.br/ws/{cep}/json/').json()

In [ ]:
agent = initialize_agent(
    load_tools([], llm=llm) + [cep],
    llm,
    agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

In [ ]:
agent("qual é o endereço completo do CEP 95020-183?")